Feature Engineering + modelowanie

**Cel:** transformacja danych na podstawie wniosnów z EDA i trenig modelu autogluonem

**Plan:**
1. Preprocessing (missing, outliers, duplikaty)
2. Time features (cyclic encoding)
3. Embeddingi z tytułów
4. Agregacje per podcast/genre
5. Target encoding
6. Trening AutoGluon i analiza wyników


Import bibliotek

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import mstats
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Sprawdź dostępność SentenceTransformers
try:
    from sentence_transformers import SentenceTransformer
    EMBEDDINGS_AVAILABLE = True
    print("SentenceTransformers dostępny!")
except ImportError:
    EMBEDDINGS_AVAILABLE = False
    print("SentenceTransformers niedostępny - użyję TF-IDF")
    from sklearn.feature_extraction.text import TfidfVectorizer

# Ścieżki
TRAIN_PATH = Path("data/train.csv")
TEST_PATH = Path("data/test.csv")

# Wczytaj dane
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Preprocessing: Flagi dla missing

**Dlaczego:**
- z eda wiemy, że braki w Guest_Popularity to około 19% a Episode_length to około 11%
- Test ma braki w tych samych miejscach, więc fagi będą pomocne

**Hipoteza:**
- Brak gościa = podcast solo -> inny wzorzec słuchalności
- model nauczy się: "Gdy Guest_Pop_missing = 1 -> przewiduj x minut

Możliwe, że to obniży RMSE o jakąś część procenta

In [ ]:
# Flagi dla Episode_Length_minutes
train["Episode_Length_missing"] = train["Episode_Length_minutes"].isnull().astype(int)
test["Episode_Length_missing"] = test["Episode_Length_minutes"].isnull().astype(int)

# Flagi dla Guest_Popularity_percentage
train["Guest_Pop_missing"] = train["Guest_Popularity_percentage"].isnull().astype(int)
test["Guest_Pop_missing"] = test["Guest_Popularity_percentage"].isnull().astype(int)

print("Flagi dodane!")
print(f"Train - Episode_Length_missing: {train['Episode_Length_missing'].sum()}")
print(f"Test - Episode_Length_missing: {test['Episode_Length_missing'].sum()}")
print(f"Train - Guest_Pop_missing: {train['Guest_Pop_missing'].sum()}")
print(f"Test - Guest_Pop_missing: {test['Guest_Pop_missing'].sum()}")

Usunięcie braków w number_of_ads (tylko train) bo jest jeden brak xd

In [ ]:
print(f"Train przed usunięciem: {len(train)}")
train = train.dropna(subset=["Number_of_Ads"])
print(f"Train po usunięciu: {len(train)}")
print(f"Usunięto: {1} wiersz")

Imputacja brakujących wartośći

**Strategia z EDA:**
- Episode_Length_minutes: mediana - prawostronna skośność
- Guest_Popularity_percentage: średnia - rozkład stmetryczny

In [ ]:
# Oblicz statystyki na train
ep_len_median = train["Episode_Length_minutes"].median()
guest_pop_mean = train["Guest_Popularity_percentage"].mean()

print(f"Episode_Length median: {ep_len_median:.2f}")
print(f"Guest_Popularity mean: {guest_pop_mean:.2f}")

# Wypełnij braki
train["Episode_Length_minutes"].fillna(ep_len_median, inplace=True)
test["Episode_Length_minutes"].fillna(ep_len_median, inplace=True)

train["Guest_Popularity_percentage"].fillna(guest_pop_mean, inplace=True)
test["Guest_Popularity_percentage"].fillna(guest_pop_mean, inplace=True)

print("\nImputation zakończony!")
print(f"Train braki: {train.isnull().sum().sum()}")
print(f"Test braki: {test.isnull().sum().sum()}")

Wywalanie outlierów

**Dlaczego?**
- bo jest ich kilka sztuk
- mają duzy wpływ na RMSE

Moze obnizyc RMSE

In [ ]:
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    before = df.shape[0]
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    after = df.shape[0]

    print(f"{col}: usunięto {before - after} outlierów (pozostało {after})")
    return df

# Usuwanie outlierów dla kluczowych zmiennych
train_clean = remove_outliers_iqr(train, "Episode_Length_minutes")
train_clean = remove_outliers_iqr(train_clean, "Number_of_Ads")

# Dopasowanie testu do zakresu treningu (opcjonalne clipping)
Q1_ep, Q3_ep = train_clean["Episode_Length_minutes"].quantile([0.25, 0.75])
IQR_ep = Q3_ep - Q1_ep
ep_lower, ep_upper = Q1_ep - 1.5 * IQR_ep, Q3_ep + 1.5 * IQR_ep

Q1_ads, Q3_ads = train_clean["Number_of_Ads"].quantile([0.25, 0.75])
IQR_ads = Q3_ads - Q1_ads
ads_lower, ads_upper = Q1_ads - 1.5 * IQR_ads, Q3_ads + 1.5 * IQR_ads

test["Episode_Length_minutes"] = test["Episode_Length_minutes"].clip(ep_lower, ep_upper)
test["Number_of_Ads"] = test["Number_of_Ads"].clip(ads_lower, ads_upper)

Usunięcie duplikatów (tylko train, zeby test mial dobra ilosc wierszy)

In [ ]:
print(f"Train przed usunięciem duplikatów: {len(train)}")
train = train.drop_duplicates()
print(f"Train po usunięciu duplikatów: {len(train)}")
print(f"Test (bez zmian): {len(test)} wierszy")

# WALIDACJA - test musi mieć 250k wierszy!
assert len(test) == 250000, f"Test ma {len(test)}, powinien mieć 250000!"
print("Test ma poprawną liczbę wierszy")

Parsowanie czasu publikacji -> numery -> cyclic encoding

In [ ]:
# Mapowanie dni tygodnia
day_mapping = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3,
    'Friday': 4, 'Saturday': 5, 'Sunday': 6
}

# Mapowanie pory dnia na godziny (środek przedziału)
time_mapping = {
    'Morning': 8, # 6-12h
    'Afternoon': 14, # 12-18h
    'Evening': 20, # 18-24h
    'Night': 2  # 24-6h (środek nocy)
}

for df in [train, test]:
    # Zamiana na numery
    df["day_of_week_num"] = df["Publication_Day"].map(day_mapping)
    df["hour_num"] = df["Publication_Time"].map(time_mapping)
    
    # CYCLIC ENCODING dla dnia tygodnia
    df["day_sin"] = np.sin(2 * np.pi * df["day_of_week_num"] / 7)
    df["day_cos"] = np.cos(2 * np.pi * df["day_of_week_num"] / 7)
    
    # CYCLIC ENCODING dla godziny
    df["hour_sin"] = np.sin(2 * np.pi * df["hour_num"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour_num"] / 24)
    
    # Binarne features
    df["is_weekend"] = (df["day_of_week_num"] >= 5).astype(int)
    df["is_morning"] = (df["hour_num"] >= 6) & (df["hour_num"] < 12)
    df["is_afternoon"] = (df["hour_num"] >= 12) & (df["hour_num"] < 18)
    df["is_evening"] = (df["hour_num"] >= 18) & (df["hour_num"] < 24)
    df["is_night"] = (df["hour_num"] < 6) | (df["hour_num"] >= 22)
    df["is_primetime"] = (df["hour_num"] >= 17) & (df["hour_num"] <= 21)

print("Parsowanie czasu zakończone!")
print(f"Przykładowe wartości (train):")
print(train[['Publication_Day', 'day_of_week_num', 'day_sin', 'day_cos']].head(3))
print(train[['Publication_Time', 'hour_num', 'hour_sin', 'hour_cos']].head(3))

Feature engineering - Podstawowe interakcje

**Dlaczego te features:**
- z EDA wiemy, że Episode_length ma najsilniejszą korelację z targetem
- Popularity (host + guest) też jest istotne
- interakcje mogą wychwycić nieliniowe zależności

Może Dalej zmniejszymy RMSE

In [ ]:
for df in [train, test]:
    # === Interakcje numeryczne ===
    df["ads_per_minute"] = df["Number_of_Ads"] / (df["Episode_Length_minutes"] + 1)
    df["total_popularity"] = df["Host_Popularity_percentage"] + df["Guest_Popularity_percentage"]
    df["popularity_ratio"] = df["Host_Popularity_percentage"] / (df["Guest_Popularity_percentage"] + 1)
    df["popularity_diff"] = df["Host_Popularity_percentage"] - df["Guest_Popularity_percentage"]
    
    # === Interakcje z flagami missing ===
    # Jeśli brak gościa, popularność hosta jest WAŻNIEJSZA
    df["missing_guest_x_host_pop"] = df["Guest_Pop_missing"] * df["Host_Popularity_percentage"]
    df["missing_length_x_ads"] = df["Episode_Length_missing"] * df["Number_of_Ads"]
    df["missing_guest_x_episode_length"] = df["Guest_Pop_missing"] * df["Episode_Length_minutes"]
    
    # === Interakcje czasowe ===
    # Wieczór + długi odcinek = więcej słuchania?
    df["length_x_evening"] = df["Episode_Length_minutes"] * df["is_evening"].astype(int)
    df["host_pop_x_weekend"] = df["Host_Popularity_percentage"] * df["is_weekend"]
    df["ads_x_primetime"] = df["Number_of_Ads"] * df["is_primetime"].astype(int)
    
    # === Sentiment jako numeric ===
    sentiment_map = {"positive": 1, "neutral": 0, "negative": -1}
    df["sentiment_numeric"] = df["Episode_Sentiment"].map(sentiment_map).fillna(0)
    
    df["sentiment_x_guest_pop"] = df["sentiment_numeric"] * df["Guest_Popularity_percentage"]
    df["sentiment_x_host_pop"] = df["sentiment_numeric"] * df["Host_Popularity_percentage"]
    df["negative_sentiment_x_ads"] = (df["sentiment_numeric"] == -1).astype(int) * df["Number_of_Ads"]

print(f"Podstawowe features dodane! Train shape: {train.shape}")

Embeddingi z Episode_Title

**Dlaczego embeddingi > text features:**
- generalnie poprzednie wyznaczniki były bez sensu (długość, liczba słów, ...)
- Embeddingi - model rozumie treść

**Metoda:**
1. SentenceTransformer (all-MiniLM-L6-v2) - szybki, 384-wymiarowy
2. PCA → redukcja do 50 wymiarów (żeby nie przeciążyć modelu)
3. KMeans clustering → grupowanie podobnych tytułów

In [ ]:
if EMBEDDINGS_AVAILABLE:
    print("Generowanie embeddingów z SentenceTransformers...")
    print("To może potrwać 3-5 minut...")
    
    # Model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generuj embeddingi dla train
    train_titles = train["Episode_Title"].fillna("").tolist()
    train_embeddings = model.encode(train_titles, show_progress_bar=True, batch_size=256)
    
    # Generuj embeddingi dla test
    test_titles = test["Episode_Title"].fillna("").tolist()
    test_embeddings = model.encode(test_titles, show_progress_bar=True, batch_size=256)
    
    print(f"Embeddingi wygenerowane! Shape: {train_embeddings.shape}")
    
    # PCA - redukcja do 50 wymiarów
    pca = PCA(n_components=50, random_state=42)
    train_embeddings_pca = pca.fit_transform(train_embeddings)
    test_embeddings_pca = pca.transform(test_embeddings)
    
    print(f"PCA zakończone! Explained variance: {pca.explained_variance_ratio_.sum():.2%}")
    
    # Dodaj jako features
    for i in range(50):
        train[f"title_emb_{i}"] = train_embeddings_pca[:, i]
        test[f"title_emb_{i}"] = test_embeddings_pca[:, i]
    
    # KMeans clustering - grupowanie podobnych tytułów
    n_clusters = 20  # 20 klastrów tematycznych
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    train["title_cluster"] = kmeans.fit_predict(train_embeddings_pca)
    test["title_cluster"] = kmeans.predict(test_embeddings_pca)
    
    print(f"Clustering zakończony! {n_clusters} klastrów")
    print(f"Rozkład klastrów (train):\n{train['title_cluster'].value_counts().head()}")
    
else:
    # Fallback: TF-IDF
    print("Używam TF-IDF jako alternatywy...")
    
    tfidf = TfidfVectorizer(max_features=50, stop_words='english', ngram_range=(1, 2))
    
    train_tfidf = tfidf.fit_transform(train["Episode_Title"].fillna(""))
    test_tfidf = tfidf.transform(test["Episode_Title"].fillna(""))
    
    # Dodaj jako features
    for i in range(50):
        train[f"title_tfidf_{i}"] = train_tfidf[:, i].toarray().flatten()
        test[f"title_tfidf_{i}"] = test_tfidf[:, i].toarray().flatten()
    
    print(f"TF-IDF zakończone! {train_tfidf.shape[1]} features")


Frequency Encoding

**Dlaczego?**
- z EDA wiemy, że niektóre podcasty mają po 10 odcinków, inne tysiące
- Popularność podcastu zapewne może korelować ze słuchalnością

In [ ]:
# Podcast frequency
podcast_freq = train["Podcast_Name"].value_counts()
train["podcast_frequency"] = train["Podcast_Name"].map(podcast_freq).fillna(0)
test["podcast_frequency"] = test["Podcast_Name"].map(podcast_freq).fillna(0)

# Genre frequency
genre_freq = train["Genre"].value_counts()
train["genre_frequency"] = train["Genre"].map(genre_freq).fillna(0)
test["genre_frequency"] = test["Genre"].map(genre_freq).fillna(0)

# Normalizacja (0-1), żeby modele regresyjne lepiej działały
train["podcast_frequency_norm"] = train["podcast_frequency"] / len(train)
test["podcast_frequency_norm"] = test["podcast_frequency"] / len(train)

train["genre_frequency_norm"] = train["genre_frequency"] / len(train)
test["genre_frequency_norm"] = test["genre_frequency"] / len(train)

print("Frequency encoding zakończony!")
print(f"Top 5 podcasts by frequency:\n{podcast_freq.head()}")

Agregacje per podcast

**Dlaczego:**
- z EDA: top podcasty mają różną średnie słuchalności (40-50 minut)

**Features:**
1. podcast_avg_listening - średnia historyczna
2. podcast_std_listening - stabilność (niski std = stała publiczność)
3. podcast_min/max_listening - zakres wartości

In [ ]:
# Statystyki per Podcast (obliczane TYLKO na train!)
podcast_stats = train.groupby("Podcast_Name").agg({
    "Listening_Time_minutes": ["mean", "median", "std", "min", "max"],
    "Episode_Length_minutes": ["mean", "median"],
    "Number_of_Ads": ["mean", "median"],
    "Host_Popularity_percentage": "first",
    "Guest_Popularity_percentage": "mean"
}).reset_index()

# Spłaszcz kolumny
podcast_stats.columns = [
    "Podcast_Name",
    "podcast_avg_listening", "podcast_med_listening", "podcast_std_listening",
    "podcast_min_listening", "podcast_max_listening",
    "podcast_avg_length", "podcast_med_length",
    "podcast_avg_ads", "podcast_med_ads",
    "podcast_host_pop", "podcast_avg_guest_pop"
]

# Wypełnij std NaN (podcasty z 1 odcinkiem)
podcast_stats["podcast_std_listening"].fillna(0, inplace=True)

# Merguj z LEFT join (ważne!)
print(f"Train przed merge: {len(train)}")
print(f"Test przed merge: {len(test)}")

train = train.merge(podcast_stats, on="Podcast_Name", how="left")
test = test.merge(podcast_stats, on="Podcast_Name", how="left")

print(f"Train po merge: {len(train)}")
print(f"Test po merge: {len(test)}")

# WALIDACJA
assert len(test) == 250000, f"Test stracił wiersze! Ma {len(test)}"
print("Merge nie stracił wierszy")

print(f"\nPodcast stats dodane! Train shape: {train.shape}")

Agregacje per Genre

**Dlaczego:**
- z eda: różnice średnich między gatunkami (np. Comedy vs News)
- Genre characteristics wpływa na słuchalność

In [ ]:
# Statystyki per Genre
genre_stats = train.groupby("Genre").agg({
    "Listening_Time_minutes": ["mean", "median", "std"],
    "Guest_Popularity_percentage": "mean",
    "Episode_Length_minutes": "mean"
}).reset_index()

genre_stats.columns = [
    "Genre",
    "genre_avg_listening", "genre_med_listening", "genre_std_listening",
    "genre_avg_guest_pop", "genre_avg_length"
]

genre_stats["genre_std_listening"].fillna(0, inplace=True)

# LEFT join
print(f"Test przed merge: {len(test)}")
train = train.merge(genre_stats, on="Genre", how="left")
test = test.merge(genre_stats, on="Genre", how="left")
print(f"Test po merge: {len(test)}")

# WALIDACJA
assert len(test) == 250000, f"Test stracił wiersze!"
print("Merge OK")

print(f"Genre stats dodane! Train shape: {train.shape}")

Relative features (odcinek vs średnia)

**Dlaczego:**
- mało liczą się wartości bezwzględne, bardziej relatywne
- np: 60-minutowy odcinek w podcaście o średniej 30 min jest wyjątkowy
- To samo 60 min w podcaście o średniej 90 min -> krótki

In [ ]:
for df in [train, test]:
    # Porównanie z podcast
    df["length_vs_podcast_avg"] = df["Episode_Length_minutes"] / (df["podcast_avg_length"] + 1)
    df["ads_vs_podcast_avg"] = df["Number_of_Ads"] / (df["podcast_avg_ads"] + 1)
    df["guest_pop_vs_podcast_avg"] = df["Guest_Popularity_percentage"] / (df["podcast_avg_guest_pop"] + 1)
    
    # Porównanie z genre
    df["length_vs_genre_avg"] = df["Episode_Length_minutes"] / (df["genre_avg_length"] + 1)
    
    # Czy ten odcinek jest powyżej/poniżej średniej?
    df["above_podcast_avg_length"] = (df["Episode_Length_minutes"] > df["podcast_avg_length"]).astype(int)
    df["above_genre_avg_length"] = (df["Episode_Length_minutes"] > df["genre_avg_length"]).astype(int)

print("Relative features dodane!")

Target Encoding

**Dlaczego:**
- High cardinality (Podcast_name: około 100 różnych wartości) - one hot nie możliwy
- Target encoding - mapowanie kategorii na średni target

Problem w postaci data leakage:
- model podgląda odpowiedzi

Mozliwe rozwiązanie: K-Fold CV
- Dla każdego fold: obliczamy średnią na pozostałych foldach
- Bayesian smoothing, regularycacja dla rzadkich kategorii

In [ ]:
def target_encode_with_cv(train_df, test_df, cat_col, target_col, n_splits=5, smoothing=10):
    """
    Target encoding z K-Fold CV (leak protection) + Bayesian smoothing
    
    Parametry:
    - smoothing: im wyższy, tym bardziej zbliżamy się do global_mean dla rzadkich kategorii
    """
    global_mean = train_df[target_col].mean()
    
    # Inicjalizacja
    train_df[f"{cat_col}_target_enc"] = global_mean
    
    # K-Fold CV dla train
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for train_idx, val_idx in kf.split(train_df):
        train_fold = train_df.iloc[train_idx]
        
        # Oblicz statystyki na foldzie treningowym
        agg = train_fold.groupby(cat_col)[target_col].agg(['mean', 'count'])
        
        # Bayesian smoothing: (count * mean + smoothing * global_mean) / (count + smoothing)
        smoothed = (agg['count'] * agg['mean'] + smoothing * global_mean) / (agg['count'] + smoothing)
        
        # Mapuj na fold walidacyjny
        train_df.loc[val_idx, f"{cat_col}_target_enc"] = train_df.loc[val_idx, cat_col].map(smoothed).fillna(global_mean)
    
    # Dla test użyj całego train
    agg_full = train_df.groupby(cat_col)[target_col].agg(['mean', 'count'])
    smoothed_full = (agg_full['count'] * agg_full['mean'] + smoothing * global_mean) / (agg_full['count'] + smoothing)
    test_df[f"{cat_col}_target_enc"] = test_df[cat_col].map(smoothed_full).fillna(global_mean)
    
    return train_df, test_df

# Zastosuj target encoding
print("Target encoding w toku (może potrwać ze 2 minuty)...")

for col in ["Podcast_Name", "Genre"]:
    train, test = target_encode_with_cv(train, test, col, "Listening_Time_minutes", smoothing=10)
    print(f"{col} zakończony")

print(f"\nTarget encoding zakończony! Train shape: {train.shape}")